In [1]:
%cd E:\kaggle\iceberg

E:\kaggle\iceberg


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from os.path import join as opj
import keras

Using TensorFlow backend.


In [3]:
train = pd.read_json("E:/kaggle/iceberg/train.json/data/processed/train.json")
target_train=train['is_iceberg']
test = pd.read_json("E:/kaggle/iceberg/test.json/data/processed/test.json")

In [4]:
train['inc_angle']=pd.to_numeric(train['inc_angle'], errors='coerce')#We have only 133 NAs.
test['inc_angle']=pd.to_numeric(test['inc_angle'], errors='coerce')
train['inc_angle']=train['inc_angle'].fillna(method='pad')
test['inc_angle']=test['inc_angle'].fillna(method='pad')
X_angle=train['inc_angle']
X_test_angle=test['inc_angle']

In [5]:
#Generate the training data
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_band_3=np.fabs(np.subtract(X_band_1,X_band_2))
X_band_4=np.maximum(X_band_1,X_band_2)
X_band_5=np.minimum(X_band_1,X_band_2)
X_train = np.concatenate([X_band_3[:, :, :, np.newaxis],X_band_4[:, :, :, np.newaxis],X_band_5[:, :, :, np.newaxis]], axis=-1)

X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_band_test_3=np.fabs(np.subtract(X_band_test_1,X_band_test_2))
X_band_test_4=np.maximum(X_band_test_1,X_band_test_2)
X_band_test_5=np.minimum(X_band_test_1,X_band_test_2)
X_test = np.concatenate([X_band_test_3[:, :, :, np.newaxis], X_band_test_4[:, :, :, np.newaxis],X_band_test_5[:, :, :, np.newaxis]],axis=-1)

In [6]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.optimizers import rmsprop
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

from keras.datasets import cifar10
from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

#Data Aug for multi-input
from keras.preprocessing.image import ImageDataGenerator

In [7]:
batch_size=64
# Define the image transformations here
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.,
                         height_shift_range = 0.,
                         channel_shift_range=0,
                         zoom_range = 0.5,
                         rotation_range = 10)

# Here is the function that merges our two generators
# We use the exact same generator with the same random seed for both the y and angle arrays
def gen_flow_for_two_inputs(X1, X2, y):
    genX1 = gen.flow(X1,y,  batch_size=batch_size,seed=55)
    genX2 = gen.flow(X1,X2, batch_size=batch_size,seed=55)
    while True:
            X1i = genX1.next()
            X2i = genX2.next()
            #Assert arrays are equal - this was for peace of mind, but slows down training
            #np.testing.assert_array_equal(X1i[0],X2i[0])
            yield [X1i[0], X2i[1]], X1i[1]

# Finally create generator
def get_callbacks(filepath, patience=2):
   #es = EarlyStopping('val_loss', patience=10, mode="min")
   es = EarlyStopping('val_loss', patience=20, mode="min")
   msave = ModelCheckpoint(filepath, save_best_only=True)
   return [es, msave]


def getVggAngleModel():
    input_2 = Input(shape=[1], name="angle")
    angle_layer = Dense(1, )(input_2)
    base_model = VGG16(weights='imagenet', include_top=False, 
                 input_shape=X_train.shape[1:], classes=1)
    x = base_model.get_layer('block5_pool').output
    x = GlobalMaxPooling2D()(x)
    base_model2 = keras.applications.mobilenet.MobileNet(weights=None, alpha=0.9,input_tensor = base_model.input,include_top=False, input_shape=X_train.shape[1:])

    x2 = base_model2.output
    x2 = GlobalAveragePooling2D()(x2)

    merge_one = concatenate([x, x2, angle_layer])

    merge_one = Dropout(0.6)(merge_one)
    predictions = Dense(1, activation='sigmoid',kernel_initializer='he_normal')(merge_one)
    
    model = Model(input=[base_model.input, input_2], output=predictions)
    
    sgd = Adam(lr=1e-4) #SGD(lr=1e-4, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model


#Using K-fold Cross Validation with Data Augmentation.
def myAngleCV(X_train, X_angle, X_test):
    K=5
    folds = list(StratifiedKFold(n_splits=K, shuffle=True, random_state=16).split(X_train, target_train))
    y_test_pred_log = 0
    y_train_pred_log=0
    y_valid_pred_log = 0.0*target_train
    for j, (train_idx, test_idx) in enumerate(folds):
        print('\n===================FOLD=',j)
        X_train_cv = X_train[train_idx]
        y_train_cv = target_train[train_idx]
        X_holdout = X_train[test_idx]
        Y_holdout= target_train[test_idx]
        
        #Angle
        X_angle_cv=X_angle[train_idx]
        X_angle_hold=X_angle[test_idx]

        #define file path and get callbacks
        file_path = "%s_aug_model_weights.hdf5"%j
        callbacks = get_callbacks(filepath=file_path, patience=10)
        gen_flow = gen_flow_for_two_inputs(X_train_cv, X_angle_cv, y_train_cv)
        galaxyModel= getVggAngleModel()
        galaxyModel.fit_generator(
                gen_flow,
                steps_per_epoch=24,
                epochs=100,
                shuffle=True,
                verbose=1,
                validation_data=([X_holdout,X_angle_hold], Y_holdout),
                callbacks=callbacks)

        #Getting the Best Model
        galaxyModel.load_weights(filepath=file_path)
        #Getting Training Score
        score = galaxyModel.evaluate([X_train_cv,X_angle_cv], y_train_cv, verbose=0)
        print('Train loss:', score[0])
        print('Train accuracy:', score[1])
        #Getting Test Score
        score = galaxyModel.evaluate([X_holdout,X_angle_hold], Y_holdout, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

        #Getting validation Score.
        pred_valid=galaxyModel.predict([X_holdout,X_angle_hold])
        y_valid_pred_log[test_idx] = pred_valid.reshape(pred_valid.shape[0])

        #Getting Test Scores
        temp_test=galaxyModel.predict([X_test, X_test_angle])
        y_test_pred_log+=temp_test.reshape(temp_test.shape[0])

        #Getting Train Scores
        temp_train=galaxyModel.predict([X_train, X_angle])
        y_train_pred_log+=temp_train.reshape(temp_train.shape[0])

    y_test_pred_log=y_test_pred_log/K
    y_train_pred_log=y_train_pred_log/K

    print('\n Train Log Loss Validation= ',log_loss(target_train, y_train_pred_log))
    print(' Test Log Loss Validation= ',log_loss(target_train, y_valid_pred_log))
    return y_test_pred_log

In [8]:
preds=myAngleCV(X_train, X_angle, X_test)
#Submission for each day.
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=preds
submission.to_csv('submission.csv', index=False)


===================FOLD= 0


D:\Miniconda3\lib\site-packages\ipykernel_launcher.py:48: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`


Epoch 1/100
24/24 [==============================] - 24s - loss: 0.7263 - acc: 0.5700 - val_loss: 0.6880 - val_acc: 0.4752
Epoch 2/100
24/24 [==============================] - 7s - loss: 0.6480 - acc: 0.6165 - val_loss: 0.6819 - val_acc: 0.7329
Epoch 3/100
24/24 [==============================] - 8s - loss: 0.6601 - acc: 0.6076 - val_loss: 0.6054 - val_acc: 0.7391
Epoch 4/100
24/24 [==============================] - 7s - loss: 0.5985 - acc: 0.6546 - val_loss: 0.5320 - val_acc: 0.7484
Epoch 5/100
24/24 [==============================] - 8s - loss: 0.6200 - acc: 0.6870 - val_loss: 0.5227 - val_acc: 0.7360
Epoch 6/100
24/24 [==============================] - 7s - loss: 0.5815 - acc: 0.6709 - val_loss: 0.4288 - val_acc: 0.7329
Epoch 7/100
24/24 [==============================] - 9s - loss: 0.4604 - acc: 0.7766 - val_loss: 0.3505 - val_acc: 0.8540
Epoch 8/100
24/24 [==============================] - 8s - loss: 0.4602 - acc: 0.7698 - val_loss: 0.3033 - val_acc: 0.8540
Epoch 9/100
24/24 [====

24/24 [==============================] - 6s - loss: 0.3501 - acc: 0.8497 - val_loss: 0.2344 - val_acc: 0.9252
Epoch 17/100
24/24 [==============================] - 6s - loss: 0.3444 - acc: 0.8657 - val_loss: 0.2052 - val_acc: 0.9128
Epoch 18/100
24/24 [==============================] - 6s - loss: 0.3327 - acc: 0.8618 - val_loss: 0.3467 - val_acc: 0.8660
Epoch 19/100
24/24 [==============================] - 6s - loss: 0.3338 - acc: 0.8631 - val_loss: 0.2569 - val_acc: 0.8941
Epoch 20/100
24/24 [==============================] - 6s - loss: 0.3621 - acc: 0.8631 - val_loss: 0.2808 - val_acc: 0.8754
Epoch 21/100
24/24 [==============================] - 7s - loss: 0.4398 - acc: 0.8135 - val_loss: 0.2096 - val_acc: 0.9065
Epoch 22/100
24/24 [==============================] - 6s - loss: 0.3667 - acc: 0.8455 - val_loss: 0.2189 - val_acc: 0.9034
Epoch 23/100
24/24 [==============================] - 6s - loss: 0.3477 - acc: 0.8696 - val_loss: 0.8286 - val_acc: 0.6760
Epoch 24/100
24/24 [=========

24/24 [==============================] - 6s - loss: 0.2421 - acc: 0.9035 - val_loss: 0.2214 - val_acc: 0.9065
Epoch 47/100
24/24 [==============================] - 6s - loss: 0.3027 - acc: 0.8660 - val_loss: 0.1923 - val_acc: 0.9252
Epoch 48/100
24/24 [==============================] - 6s - loss: 0.2554 - acc: 0.9002 - val_loss: 0.1958 - val_acc: 0.9159
Epoch 49/100
24/24 [==============================] - 7s - loss: 0.2712 - acc: 0.8759 - val_loss: 0.3827 - val_acc: 0.8629
Epoch 50/100
24/24 [==============================] - 6s - loss: 0.3384 - acc: 0.8471 - val_loss: 0.1924 - val_acc: 0.9439
Epoch 51/100
24/24 [==============================] - 7s - loss: 0.2721 - acc: 0.8745 - val_loss: 0.1684 - val_acc: 0.9283
Epoch 52/100
24/24 [==============================] - 6s - loss: 0.2275 - acc: 0.9002 - val_loss: 0.2626 - val_acc: 0.9065
Epoch 53/100
24/24 [==============================] - 6s - loss: 0.2624 - acc: 0.8856 - val_loss: 0.1935 - val_acc: 0.9346
Epoch 54/100
24/24 [=========

24/24 [==============================] - 6s - loss: 0.2331 - acc: 0.8989 - val_loss: 0.3965 - val_acc: 0.8406
Epoch 21/100
24/24 [==============================] - 6s - loss: 0.2414 - acc: 0.8955 - val_loss: 0.3990 - val_acc: 0.8187
Epoch 22/100
24/24 [==============================] - 6s - loss: 0.2493 - acc: 0.8847 - val_loss: 0.4025 - val_acc: 0.8313
Epoch 23/100
24/24 [==============================] - 6s - loss: 0.2727 - acc: 0.8904 - val_loss: 0.3876 - val_acc: 0.8156
Epoch 24/100
24/24 [==============================] - 7s - loss: 0.2524 - acc: 0.8898 - val_loss: 0.3324 - val_acc: 0.8375
Epoch 25/100
24/24 [==============================] - 8s - loss: 0.2306 - acc: 0.8983 - val_loss: 0.3196 - val_acc: 0.8313
Epoch 26/100
24/24 [==============================] - 6s - loss: 0.2407 - acc: 0.9041 - val_loss: 0.3253 - val_acc: 0.8344
Epoch 27/100
24/24 [==============================] - 6s - loss: 0.2448 - acc: 0.9013 - val_loss: 0.4676 - val_acc: 0.8344
Epoch 28/100
24/24 [=========

24/24 [==============================] - 6s - loss: 0.3337 - acc: 0.8487 - val_loss: 0.3735 - val_acc: 0.8063
Epoch 12/100
24/24 [==============================] - 6s - loss: 0.3163 - acc: 0.8468 - val_loss: 0.3006 - val_acc: 0.8656
Epoch 13/100
24/24 [==============================] - 8s - loss: 0.2802 - acc: 0.8748 - val_loss: 0.2746 - val_acc: 0.8625
Epoch 14/100
24/24 [==============================] - 7s - loss: 0.2997 - acc: 0.8606 - val_loss: 0.2909 - val_acc: 0.8688
Epoch 15/100
24/24 [==============================] - 6s - loss: 0.2737 - acc: 0.8754 - val_loss: 0.3547 - val_acc: 0.8375
Epoch 16/100
24/24 [==============================] - 6s - loss: 0.3437 - acc: 0.8426 - val_loss: 0.4114 - val_acc: 0.7250
Epoch 17/100
24/24 [==============================] - 6s - loss: 0.3578 - acc: 0.8393 - val_loss: 0.3781 - val_acc: 0.8531
Epoch 18/100
24/24 [==============================] - 6s - loss: 0.3377 - acc: 0.8578 - val_loss: 0.3047 - val_acc: 0.8531
Epoch 19/100
24/24 [=========